In [1]:
import numpy as np
import torch

from envs.IBGym_mod_envs import IBGymQ
from state_quantization.transforms import NormalizeTransform
from dynamic_programming.mdp_model import DPPolicy
import matplotlib.pyplot as plt
from envs.env_creator import env_creator

In [2]:


device = 'cpu'


env = env_creator(config=None,)

policy_save_path = 'dynamic_programming/ib_policy.pkl'
policy = DPPolicy.load(policy_save_path)

<class 'state_quantization.quantization_models.ForcastingDiscFinalState'>


/home/hamza/PycharmProjects/StateCompression/venv/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [3]:
policy.policy_table

array([ 0,  0, 19, 17,  1,  8, 19, 10,  0,  7,  5,  6, 22, 17, 26, 12, 25,
       26, 12, 24,  0,  6, 16, 18,  7, 15,  5,  1,  0,  3,  6,  8,  3, 14,
        1,  1,  7, 11, 18, 17, 11, 23, 16,  0, 25, 12, 16, 23, 13, 16,  1,
       24, 17, 23, 24, 14, 24,  4, 17, 24, 23, 12, 21, 24,  7,  8,  3, 17,
        8, 21, 16, 19, 25,  6,  6,  3, 23, 14,  5,  8, 25,  4, 11,  7, 26,
       24, 26, 26, 19, 16, 24,  8, 26, 24,  5,  6, 22, 24, 15, 26,  5,  3,
       26,  6, 26, 26, 24, 12, 25, 22, 15, 11, 19,  0,  9,  9, 22,  0,  0,
       18,  0,  0,  0,  1,  0,  0, 18,  9, 23,  0, 25,  4, 15,  5,  5,  7,
        1,  0,  0,  9,  0, 10,  1,  2,  1,  7, 16, 24,  0, 18,  9, 23,  1,
        7, 25, 26,  9,  3, 25, 26,  6, 18, 12, 10,  7,  6, 16, 17, 25,  4,
       11,  7, 22, 25, 26, 24,  8, 11, 14,  9, 10,  5,  5,  0,  2, 23, 16,
        6, 24,  0,  9,  0,  5, 17, 25, 26, 25, 23,  2,  1, 25, 26, 20,  7,
        5,  0,  1,  8,  7,  3, 23,  8,  6, 24,  2,  1,  4, 24, 24, 25, 26,
       25,  4, 24, 25, 15

In [ ]:
total_epochs = 0
episodes = 25
avg_ep_reward = 0

without_random_action_rewards = []
with_random_action_rewards = []
states_each_ep = []
actions_each_ep = []

for i in range(episodes):
    print(f'Episode: {i}')
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0

    done = False
    total_rewards = 0
    random_action = False

    #states_enc = set()
    states = []
    actions = []
    while not done:
        states.append(state)
        if state in policy.state_to_index:
            action = policy.get_action(state)
        else:
            action = env.action_space.sample()
            random_action = True
        actions.append(action)
        state, reward, done, info = env.step(action)


        epochs += 1

        total_rewards += reward
    if random_action:
        with_random_action_rewards.append(total_rewards/env.reset_after_timesteps)
        print('random action taken')
    else:
        without_random_action_rewards.append(total_rewards/env.reset_after_timesteps)
    states_each_ep.append(states)
    print((total_rewards/env.reset_after_timesteps))
    avg_ep_reward += (total_rewards/env.reset_after_timesteps)
    total_epochs += epochs
    print(f'States this episode: {np.unique(states_each_ep[-1]).size}')

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average Reward per episode: {avg_ep_reward / episodes}")


Episode: 0
-222.20144542303464
States this episode: 8
Episode: 1
-222.26005112720406
States this episode: 8
Episode: 2
-222.01259670672863
States this episode: 7
Episode: 3
-221.9438806830762
States this episode: 7
Episode: 4
-222.08005911035312
States this episode: 7
Episode: 5
-222.3863280557885
States this episode: 10
Episode: 6
-221.7510941686463
States this episode: 6
Episode: 7
-222.8336325545699
States this episode: 7
Episode: 8
-221.85132572500402
States this episode: 7
Episode: 9


In [ ]:
#print(f'Average Reward with random actions {sum(with_random_action_rewards)/len(with_random_action_rewards)}')
print(f'Average Reward without random actions {sum(without_random_action_rewards)/len(without_random_action_rewards)}')

In [ ]:
without_random_action_rewards

In [ ]:
un, inv = np.unique(states_each_ep,return_inverse=True)
inv = np.array(np.split(inv, len(states_each_ep)))

In [ ]:
un.size

In [ ]:
for ep in inv:
    plt.plot(ep)
plt.show()

In [ ]:
for ep in inv:
    plt.plot(actions)
plt.show()

In [ ]:
np.unique(actions).size